In [2]:
import keras
from os.path import join
import os
from bs4 import BeautifulSoup as BS
from constants import MAX_SEQUENCE_LENGTH, MAX_NUM_WORDS, EMBEDDING_DIM
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from datetime import datetime
from tqdm import tqdm
import re

np.random.seed(1337)
def replace_timezone_fn(s):
    return re.sub('(\d){2}:(\d){2}:(\d){2} [A-Z]{3}', _sub_tz, s)
def _sub_tz(matched):
    return re.sub('[A-Za-z]{3}', 'GMT', matched.group(0))
     
def retrive_date(pat_obj_1, pat_obj_2, pat_obj_3, s):
    s = s.lower()
    
    match_1 = pat_obj_1.search(s)
    match_2 = None
    match_3 = None
    
    if match_1 is None:
        match_2 = pat_obj_2.search(s)
    if match_2 is None:
        match_3 = pat_obj_3.search(s)
    assert (match_1 is not None) | (match_2 is not None) | (match_3 is not None)
    
        
    if match_1 is not None:
        days, months, years = match_1.groups()
        date_str = '%s %s %s' % (days, months, years)
    if match_2 is not None:
        days, months, years = match_2.groups()
        date_str = '%s %s 19%s' % (days, months, years)
    if match_3 is not None:
        months, days, years = match_3.groups()
        date_str = '%s %s %s' % (days, months, years)
    
    if len(months) > 3:
        return datetime.strptime(date_str, '%d %B %Y')
    else:
        return datetime.strptime(date_str, '%d %b %Y')
    

def prep_date(date):
#     min date in training set 1991-12-31
    secs = (date - datetime.strptime('1990-9-9', '%Y-%m-%d')).total_seconds()
    weeks = secs // (60*60*24*7)
    return weeks

def quote_title_date_abstract(xml_path):
    with open(xml_path, 'r') as f:
        data = f.read()
    soup = BS(data)
    title, abstract, date = soup.find('title').text, soup.find('abstract').text, soup.find('date').text
    
    return title.strip(), abstract.strip(), date.strip()

# text preprocessing
data_path = join('./','kaggle/')
xml_dir = join(data_path, 't3-doc')
xml_list = [f for f in os.listdir(xml_dir) if f.endswith('.xml')]
# print(len(xml_list))


texts = []
weeks = []
c = 0
# 
month_pattern = '(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|january|february|march|april|may|june|july|august|september|october|november|december)'
pat_obj_1 = re.compile('([0-9]+) %s ([0-9]{4}|20[0-9]{3})' % month_pattern)
pat_obj_2 = re.compile('([0-9]+) %s (9[0-9])' % month_pattern)
pat_obj_3 = re.compile('%s ([0-9]+) .* ([0-9]{4}|20[0-9]{3})' % month_pattern)
# 

with tqdm(total=len(xml_list)) as pbar:
    for xml in xml_list:
        
        
        path = join(xml_dir,xml)
        title, abstract, date_str = quote_title_date_abstract(path)
        text = title + '' + abstract
#         date special case
#         01/01/93 13:35:33 GMT+0100 12330.xml
        if xml == '12330.xml':
            date_str = '1 Jan 93'

        try :
            date = retrive_date(pat_obj_1, pat_obj_2, pat_obj_3, date_str)
        except AssertionError:
            print date_str, xml
            c+=1 
            continue
#             break
    #     Mon, 8 Dec 1997 16:22:29 GMT
    #     datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')
        texts.append(text)
        weeks.append(prep_date(date))
    #     texts.append(title)
    #     texts.append(abstract)
        pbar.update(1)
print('read all %d xml files.' % len(xml_list))
print 'error count %d' % c
print min(weeks)

100%|██████████| 17500/17500 [00:07<00:00, 2344.58it/s]

read all 17500 xml files.
error count 0
68.0


In [5]:
print min(dates), max(dates), np.mean(dates), np.std(dates)

68.0 659.0 395.61314285714286 163.49099703952996


In [8]:
import datetime
from pytz import timezone
import re
# month_pattern = 
s = 'thu, 26 sep 1996 15:31:53 +0200'
month_pattern = '(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)'
pat_obj_1 = re.compile('([0-9]+) %s ([0-9]{4}|20[0-9]{3})' % month_pattern)
pat_obj_2 = re.compile('([0-9]+) %s (9[0-9])' % month_pattern)
pat_obj_3 = re.compile('%s ([0-9]+) .* ([0-9]{4}|20[0-9]{3})' % month_pattern)


a = datetime.datetime.strptime('1 1 1994', '%d %m %Y')
x = ''
for i in range(13):
    b = a + datetime.timedelta(days=i*29)
    print b.time()
#     print b.ctime()[3:7]
#     x += '' +  b.ctime()[3:7]
# print x[4:]
# print months_set
# s = '('

# for m in months_set:
#     s += '%s|' %m
# s += ')'
# print s
# print pat_obj_1.search(s.lower())
# print pat_obj_1.findall(s.lower())
# print pat_obj_2.search(s.lower())
# print pat_obj_3.search(s.lower())
# print pat_obj_3.findall(s.lower())


00:00:00
00:00:00
00:00:00
00:00:00
00:00:00
00:00:00
00:00:00
00:00:00
00:00:00
00:00:00
00:00:00
00:00:00
00:00:00


In [43]:
import calendar
x = ''
for i in range(12):
    cal = calendar.month(2016, i+1)
# print "以下输出2016年1月份的日历:"
    a = cal[:12].replace('20','').replace('2','').strip()
    x += '%s|' % a
print x.lower()

january|february|march|april|may|june|july|august|september|october|november|december|


In [49]:
a = '1997-06-12 00:00:00'
b = '1996-11-14 00:00:00'
a = datetime.strptime(a, '%Y-%m-%d 00:00:00')
b = datetime.strptime(b, '%Y-%m-%d 00:00:00')
print (a-b).total_seconds()
print min(a,b)
# print a.total_seconds()

18144000.0
1996-11-14 00:00:00
